In [1]:
import iris

import numpy as np
import scipy.stats

In [2]:
era5 = iris.load("testing_data/ERA5.nc")
obs = era5[0]

cm_hist = iris.load_cube("testing_data/CMIP6/medium_area/historical_monthly_surface_temp_single_levels_IPSL-CM5A2-INCA/tas_Amon_IPSL-CM5A2-INCA_historical_r1i1p1f1_gr_19790116-20141216_v20200729.nc")
cm_future = iris.load_cube("testing_data/CMIP6/medium_area/SSP3-7_monthly_surface_temp_single_levels_IPSL-CM5A2-INCA/tas_Amon_IPSL-CM5A2-INCA_ssp370_r1i1p1f1_gr_20150116-20501216_v20201218.nc")

/home/jakobwes/anaconda3/lib/python3.8/site-packages/iris/fileformats/cf.py:859: UserWarning: Missing CF-netCDF measure variable 'areacella', referenced by netCDF variable 'tas'
  warnings.warn(
/home/jakobwes/anaconda3/lib/python3.8/site-packages/iris/fileformats/cf.py:859: UserWarning: Missing CF-netCDF measure variable 'areacella', referenced by netCDF variable 'tas'
  warnings.warn(


In [3]:
# Regrid
cm_hist = cm_hist.regrid(obs, iris.analysis.Linear())
cm_future = cm_future.regrid(obs, iris.analysis.Linear())

In [4]:
# Get data
obs = obs.data[:, 0, :, :]
cm_hist = cm_hist.data
cm_future = cm_future.data

Test different debiasers:

In [8]:
from scipy.stats import norm

from quantile_mapping import QuantileMapping
from delta_quantile_mapping import DeltaQuantileMapping
from switanek2017 import Switanek2017

distribution = norm

temp_debiaser = QuantileMapping(distribution = norm, delta_type = "none")
debiased_cm1 = temp_debiaser.apply(obs, cm_hist, cm_future)

temp_debiaser = QuantileMapping(distribution = norm, delta_type = "additive")
debiased_cm2 = temp_debiaser.apply(obs, cm_hist, cm_future)

temp_debiaser = DeltaQuantileMapping(distribution = norm, time_window_length = 50)
debiased_cm3 = temp_debiaser.apply(obs, cm_hist, cm_future)

temp_debiaser = Switanek2017()
debiased_cm3 = temp_debiaser.apply(obs, cm_hist, cm_future)


----- Running debiasing -----


100%|██████████| 357/357 [00:00<00:00, 1132.28it/s]


----- Running debiasing -----


100%|██████████| 357/357 [00:00<00:00, 1031.94it/s]


----- Running debiasing -----


100%|██████████| 357/357 [00:13<00:00, 25.76it/s]


----- Running debiasing -----


100%|██████████| 357/357 [00:00<00:00, 450.93it/s]
